# Run pyCIAM with regional rise SLR data

This notebook:
1. Converts the regional_rise folder NetCDFs (slr_ssp126/245/370/585.nc) to a pyCIAM-compatible SLR zarr.
2. Maps SLIIDERS segment coordinates (seg_lon, seg_lat) to the nearest grid points in the regional rise data via pyCIAM's spherical nearest neighbor.
3. Runs the pyCIAM model.

In [1]:
import sys
from pathlib import Path

# Repo root: notebook lives in regional_rise/, so parent is repo root
REGIONAL_RISE_DIR = Path.cwd() if (Path.cwd() / "slr_ssp126.nc").exists() else Path(__file__).resolve().parent
REPO_ROOT = REGIONAL_RISE_DIR.parent
PYCIAM_DIR = REPO_ROOT / "pyCIAM-1.1.2"
if not PYCIAM_DIR.exists():
    PYCIAM_DIR = Path.cwd() / "pyCIAM-1.1.2"
sys.path.insert(0, str(PYCIAM_DIR))
sys.path.insert(0, str(REGIONAL_RISE_DIR))  # for regional_rise_to_zarr

In [2]:
# Paths: SLIIDERS (socioeconomic), regional rise SLR zarr, params, outputs
PATH_SLIIDERS = REPO_ROOT / "sliiders-v1.2.zarr"
PATH_SLIIDERS_SEG = REGIONAL_RISE_DIR / "sliiders-v1.2-seg.zarr"
PATH_SLR_REGIONAL = REGIONAL_RISE_DIR / "slr_regional_rise.zarr"
PATH_PARAMS = PYCIAM_DIR / "params.json"
PATH_REFA = REGIONAL_RISE_DIR / "refA_regional_rise.zarr"
PATH_OUTPUTS = REGIONAL_RISE_DIR / "pyciam_outputs.zarr"
PATH_TMP = REGIONAL_RISE_DIR / "pyciam_tmp.zarr"

# No precomputed surge lookup: pass None so pyCIAM uses default and computes ESL on the fly (slower but works).
PATHS_SURGE_LOOKUP = None

## 1. Convert regional rise NetCDFs to pyCIAM SLR zarr

Builds a zarr with dimensions `(site_id, scenario, quantile, year)` so that pyCIAM can map each SLIIDERS segment to the nearest `site_id` (lat/lon) in this grid.

In [3]:
from regional_rise_to_zarr import build_slr_zarr

if not PATH_SLR_REGIONAL.exists():
    build_slr_zarr(
        input_dir=REGIONAL_RISE_DIR,
        output_path=PATH_SLR_REGIONAL,
        quantiles=[0.5],
        ncc_from_scenario="SSP126",
    )
    print(f"Built {PATH_SLR_REGIONAL}")
else:
    print(f"Using existing {PATH_SLR_REGIONAL}")

Using existing c:\Users\r2d2d\Documents\GitHub\DSC180B-B05-1-Capstone\regional_rise\slr_regional_rise.zarr


## 2. (Optional) Verify mapping: SLIIDERS segments → nearest regional rise sites

pyCIAM does this internally via `get_nearest_slrs`. Here we show the mapping for inspection.

In [4]:
import xarray as xr
from pyCIAM.io import get_nearest_slrs

sliiders = xr.open_zarr(str(PATH_SLIIDERS))
slr_zarr = xr.open_zarr(str(PATH_SLR_REGIONAL))

# Unique segment lat/lon (from seg_adm level; seg_lat/seg_lon are on seg_adm)
lonlats = sliiders[["seg_lon", "seg_lat"]].to_dataframe()
nearest_ids = get_nearest_slrs(slr_zarr, lonlats)
print("Segment → nearest SLR site_id: first 10")
print(nearest_ids.head(10))
print(f"SLIIDERS seg_adm count: {len(lonlats)}")
print(f"Regional rise site count: {slr_zarr.dims['site_id']}")

c:\Users\r2d2d\micromamba\envs\pyciam\Lib\site-packages\rhg_compute_tools\__init__.py:6: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


Segment → nearest SLR site_id: first 10
seg_adm
seg_00001_adm1_GRC.4_1     44664
seg_00002_adm1_GRC.4_1     44664
seg_00003_adm1_GRC.1_1     45026
seg_00008_adm1_ITA.15_1    45012
seg_00009_adm1_ITA.15_1    45012
seg_00010_adm1_GRC.4_1     45023
seg_00015_adm1_GRC.4_1     45024
seg_00020_adm1_GRC.4_1     45024
seg_00025_adm1_GRC.4_1     45024
seg_00030_adm1_GRC.4_1     45025
Name: ids, dtype: int64
SLIIDERS seg_adm count: 11980
Regional rise site count: 64800


## 3. Run pyCIAM

Uses the regional rise SLR zarr and SLIIDERS. Longitude/latitude mapping is done inside `load_ciam_inputs` → `get_nearest_slrs`. For a full run we use `execute_pyciam` (requires Dask). For a quick single-segment test, use the cell below instead.

In [ ]:
import pandas as pd
from distributed import Client
from pyCIAM.run import execute_pyciam

# Parameters for execute_pyciam
QUANTILES = [0.5]
SEG_CHUNKSIZE = 3
SEG_ADM_SUBSET = None  # e.g. "_USA" to restrict to US segments

client = Client( n_workers=2,
    threads_per_worker=2,
    memory_limit="3GB",)  # local Dask cluster
print(client.dashboard_link)

http://127.0.0.1:8787/status


2026-02-12 17:47:09,059 - distributed.nanny.memory - WARNING - Worker tcp://127.0.0.1:62908 (pid=17164) exceeded 95% memory budget. Restarting...
2026-02-12 17:47:10,107 - distributed.nanny - WARNING - Restarting worker
2026-02-12 17:47:12,426 - distributed.nanny.memory - WARNING - Worker tcp://127.0.0.1:62899 (pid=13404) exceeded 95% memory budget. Restarting...
2026-02-12 17:47:13,058 - distributed.nanny - WARNING - Restarting worker
2026-02-12 17:47:13,121 - distributed.nanny.memory - WARNING - Worker tcp://127.0.0.1:62898 (pid=20868) exceeded 95% memory budget. Restarting...
2026-02-12 17:47:13,314 - distributed.nanny.memory - WARNING - Worker tcp://127.0.0.1:62897 (pid=20224) exceeded 95% memory budget. Restarting...
2026-02-12 17:47:13,640 - distributed.nanny - WARNING - Restarting worker
2026-02-12 17:47:13,832 - distributed.nanny - WARNING - Restarting worker
2026-02-12 17:47:18,952 - distributed.nanny.memory - WARNING - Worker tcp://127.0.0.1:62892 (pid=27408) exceeded 95% mem

In [ ]:
execute_pyciam(
    params_path=PATH_PARAMS,
    econ_input_path=PATH_SLIIDERS, 
    slr_input_paths=[PATH_SLR_REGIONAL],
    slr_names=["regional_rise"],
    refA_path=PATH_REFA,
    econ_input_path_seg=PATH_SLIIDERS_SEG,
    output_path=PATH_OUTPUTS,
    tmp_output_path=PATH_TMP,
    surge_input_paths=PATHS_SURGE_LOOKUP,
    mc_dim="quantile",
    quantiles=QUANTILES,
    dask_client_func=lambda: client,
    refA_seg_chunksize=50,
    surge_seg_chunksize=3,
    pyciam_seg_chunksize=SEG_CHUNKSIZE,
    extra_attrs={
        "description": "pyCIAM run with regional rise SLR data mapped to SLIIDERS segments",
    },
)

KilledWorker: Attempted to run task get_refA-f60a62fc892d7a9fd321e07232ba01bd on 3 different workers, but all those workers died while running it. The last worker that attempt to run the task was tcp://127.0.0.1:49503. Inspecting worker logs is often a good next step to diagnose what went wrong. For more information see https://distributed.dask.org/en/stable/killed.html.

## Alternative: Quick single-segment test (no Dask)

Run the cell below instead of `execute_pyciam` to test one segment without the full pipeline.

In [ ]:
# Uncomment and run for a quick test on one segment:
# import json
# import xarray as xr
# from pyCIAM.io import load_ciam_inputs
# from pyCIAM.run import calc_costs, calc_all_cases
# 
# params = pd.read_json(PATH_PARAMS)["values"]
# sliiders = xr.open_zarr(str(PATH_SLIIDERS))
# one_seg = [sliiders.seg_adm.values[0]]  # first segment-admin
# inputs, slr, surge = load_ciam_inputs(
#     PATH_SLIIDERS,
#     PATH_SLR_REGIONAL,
#     params,
#     one_seg,
#     slr_names=["regional_rise"],
#     seg_var="seg_adm",
#     surge_lookup_store=None,
#     mc_dim="quantile",
#     quantiles=[0.5],
# )
# print("Inputs and SLR loaded; segment", one_seg)
# print("SLR years:", slr.year.values)
# costs = calc_costs(inputs, slr.unstack("scen_mc"), surge_lookup=surge)
# print("Costs shape:", costs.costs.shape)

: 

: 

: 